# Source Detection

 work with Weakly_2023_44
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/01/07
- update : 2024/01/09


In [ ]:
# General python packages
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# LSST Science Pipeline packages (see pipelines.lsst.io)
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

# Pipeline tasks
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (12,12)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

https://matplotlib.org/stable/users/explain/colors/colormaps.html

In [ ]:
# LSST Display
#tableau-colorblind10: #006BA4, #FF800E, #ABABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF
plt.style.use('tableau-colorblind10')
the_tableau_blindcolors10 = plt.rcParams['axes.prop_cycle'].by_key()['color'] 
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [ ]:
print("the_tableau_blindcolors10 = ", the_tableau_blindcolors10)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors

rgb_colorsList = [ matplotlib.colors.to_rgb(x) for x in the_tableau_blindcolors10 ] 

# Create the colormap
cmap_tableau_colorblind10 = LinearSegmentedColormap.from_list("my_tableau_blindcolors10_cmap", rgb_colorsList, N=10)

In [ ]:
cmap_tableau_colorblind10

In [ ]:
#import matplotlib
#cm=matplotlib.colors.Colormap('tableau-colorblind10')

In [ ]:
#cm

In [ ]:
#plt.get_cmap(cm)

In [ ]:
#cm = plt.cm.tab10('tableau-colorblind10')

#bwr_map = plt.get_cmap('bwr')
#reversed_map = bwr_map.reversed() 
#cNorm = colors.Normalize(0., vmax=1.)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
#all_colors = scalarMap.to_rgba(df["zobs"].values, alpha=1)

In [ ]:
#tab_map = plt.get_cmap('tableau-colorblind10')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

In [ ]:
butler = Butler(repo,collections=my_collection)
registry = butler.registry

## Load Visits

In [ ]:
filevisit_in = "../data/202402/ccdVisitTable_202402.csv"

In [ ]:
df = pd.read_csv(filevisit_in)

In [ ]:
df

In [ ]:
tract_selected = 5615
patch_selected = 294
nightObs_selected = 20230803

In [ ]:
cut1 = (df.nightObs == nightObs_selected) 
cut2 =  (df.tractID == tract_selected) 
cut3 = (df.patchID == patch_selected) 	

In [ ]:
cut = cut1 & cut2 & cut3

In [ ]:
df = df[cut]

In [ ]:
df

In [ ]:
exposure_id = 2023080300406
exposure_title =f"Exposure {exposure_id} (Auxtel Photometry)"

In [ ]:
# Define the dataId using just visit and detector
dataId = {'visit': exposure_id,'instrument':"LATISS", 'detector': 0}

# Use the butler to get the calexp
calexp = butler.get('calexp', **dataId, collections=my_collection)

<br>

As described in other tutorials, the `calexp` object possesses more than just the raw pixel data of the image. It also contains a `mask`, which stores information about various pixels in a bit mask.

Here are some optional commands to explore the calexp. Uncomment one of the code lines to learn more.

In [ ]:
# If you want to investigate the contents of the masked image:
#calexp.maskedImage

# If you just want one of the three components:
# calexp.maskedImage.image
# calexp.maskedImage.mask
# calexp.maskedImage.variance

# These also work:
# calexp.image
# calexp.mask
# calexp.variance

# The calexp also contains the PSF, the WCS, and the photometric calibration
#calexp.getPsf()
#calexp.getWcs()
#calexp.getPhotoCalib()

Since we are interested in performing our own source detection and measurement, we choose to clear the existing `DETECTED` mask plane.

In [ ]:
# Unset the `DETECTED` bits of the mask plane
calexp.mask.removeAndClearMaskPlane('DETECTED')

In [ ]:
plt.imshow(calexp.mask.array,origin='lower')
plt.suptitle( "Calexp Mask " + exposure_title)

In [ ]:
cmap_tableau_colorblind10

In [ ]:
calexp.mask

In [ ]:
# Plot the calexp we just retrieved
plt.figure(figsize=(14,14))
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image, title= "Original Calexp "+exposure_title)

<br>

### 2.2. Add the Subtracted Sky Background Back into the Image

Here we retrieve the subtracted background for the same dataId and add it back into the image. This section is optional.

First, we obtain the `calexpBackground` object for this `dataId`.  We will again use the `butler`.

In [ ]:
bkgd = butler.get('calexpBackground', **dataId)

Now, let us display the background we obtained.

In [ ]:
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('linear', 'zscale')
afw_display.mtv(bkgd.getImage())
plt.title("Local Polynomial Background for calexp "+ exposure_title)

In [ ]:
# Note: executing this cell multiple times will add the background
#  multiple times
calexp.maskedImage += bkgd.getImage()

Next, we add the background into the `calexp`, and re-display the `calexp`. Note the scale in the sidebar now goes up to thousands of counts instead of hundreds of counts.

In [ ]:
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image,title= "Calexp + Background "+exposure_title)

## 3. Source Detection, Deblending, and Measurement

We now want to run the LSST Science Pipelines' source detection, deblending, and measurement tasks. While we run all three tasks, this notebook is mostly focused on the detection of sources.

Recall that these tasks were imported up at the top of this notebook, from `lsst.pipe` and `lsst.meas`. More information can be found at [pipelines.lsst.io](https://pipelines.lsst.io/) (the search bar at the top left of that page is a very handy way to find documentation for a specific task).

We start by creating a minimal schema for the source table. The schema describes the output properties that will be measured for each source. This schema will be passed to all of the tasks, as we call each in turn, and each task will add columns to this schema as it measures sources in the image.

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

### 3.1. Configuring Tasks

Each task possesses an associated configuration class. The properties of these configuration classes can be determined from the classes themselves.

In [ ]:
# Uncomment the following line to view help
#  for the CharacterizeImageTask configuration
# Replace 'CharacterizeImageTask' with a different
#  task name to view additional help information

# 
#help(CharacterizeImageTask.ConfigClass())

As a starting point, like the `schema` and `algMetadata` above, here we set some basic config parameters and instantiate the tasks to get you started. In this case, we configure several different tasks:

* CharacterizeImageTask: Characterizes the image properties (e.g., PSF, etc.)
* SourceDetectionTask: Detects sources
* SourceDeblendTask: Deblend sources into constituent "children"
* SingleFrameMeasurementTask: Measures source properties

In [ ]:
# Characterize the image properties
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 5
charImageTask = CharacterizeImageTask(config=config)

# Detect sources
config = SourceDetectionTask.ConfigClass()
# detection threshold in units of thresholdType
config.thresholdValue = 20
#config.thresholdValue = 10
# units for thresholdValue
config.thresholdType = "stdev"
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

# Deblend sources
sourceDeblendTask = SourceDeblendTask(schema=schema)

# Measure source properties
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=config,
                                                   algMetadata=algMetadata)

Note that if you want to change the value of a config parameter (e.g., `psfIterations`), do not change it in the already constructed task. Instead, change your config object and then construct a new characterize image task. Like so:
> `config.psfIterations = 3` <br>
> `charImageTask = CharacterizeImageTask(config=config)`

Like the configs, we can use `help` to explore each task and the methods used to run it.

In [ ]:
# help(charImageTask)

# Uncomment the following line, position your cursor after the period,
#  and press tab to see a list of all methods. Then recomment the line
#  because "Task." is not executable and will cause an error.
# charImageTask.

# Use the help function on any of the methods to learn more:
# help(charImageTask.writeSchemas)

# E.g., find out what options there are for config.thresholdType
# help(SourceDetectionTask.ConfigClass)

With each of the tasks configured, we can now move on to running the source detection, deblending, and measurement. First we create `SourceTable` for holding the output of our source analysis. The columns and characteristics of this table are defined by the `schema` that we created in our configuration step.

In [ ]:
tab = afwTable.SourceTable.make(schema)

In [ ]:
# Image characterization (this cell may take a few seconds)
result = charImageTask.run(calexp)

# Define the pixel coordinates of a point of interest
# (in this case, basically a random point within the image)
x_target, y_target = 1700, 2100
width, height = 400, 400
xmin, ymin = x_target-width//2, y_target-height//2
point = geom.Point2D(x_target, y_target)

# Get the PSF at our point of interest
psf = calexp.getPsf()
sigma = psf.computeShape(point).getDeterminantRadius()
pixelScale = calexp.getWcs().getPixelScale().asArcseconds()

# The factor of 2.355 converts from std to fwhm
print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

In [ ]:
# Source detection (this cell may take a few seconds)
result = sourceDetectionTask.run(tab, calexp)
type(result)

With the image characterized, we are now interested in running the source detection, deblending, and measurement tasks. Each of these tasks is called with the `run` method. The parameters of this method can be investigated using `help`.

In [ ]:
# We are specifically interested in the `SourceMeasurementTask`
#help(sourceMeasurementTask.run)

In [ ]:
for k, v in result.getDict().items():
    print(k, type(v))

In [ ]:
result.numPosPeaks

In [ ]:
sources = result.sources

In [ ]:
sources.writeFits("outputTable.fits")
calexp.writeFits("example1-out.fits")

In [ ]:
# Source deblending
sourceDeblendTask.run(calexp, sources)

# Source measurement
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources_astropy = sources.copy(True)

# Investigate the output source catalog
sources_astropy = sources_astropy.asAstropy()

In [ ]:
# Define a small region for a cutout
bbox = geom.Box2I()
bbox.include(geom.Point2I(xmin, ymin))
bbox.include(geom.Point2I(xmin + width, ymin + height))

# An alternative way to defined the same cutout region
# bbox = geom.Box2I(geom.Point2I(xmin, ymin), geom.Extent2I(width, height))

# Generate the cutout image
cutout = calexp.Factory(calexp, bbox, origin=afwImage.LOCAL, deep=False)

In [ ]:
# Display the cutout and sources with afw display
#image = cutout.image
image = calexp.image
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image, title= "Source Identification on full Calexp "+ exposure_title)
#plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image
#  after each source is plotted
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('.', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=100, ctype='orange')

In [ ]:
# Display the cutout and sources with afw display
image = cutout.image
#image = calexp.image
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image, title= "Source Identification on Cutout Calexp "+ exposure_title)
plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image
#  after each source is plotted
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('.', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=100, ctype='orange')

In [ ]:
df = sources_astropy.to_pandas()

In [ ]:
df["base_PsfFlux_instMag"] =  df['base_PsfFlux_instFlux'].map(convert_fluxtomag)
#df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(lambda x:-2.50 * np.log10(x) + 31.4)
df["slot_PsfFlux_instMag"] =  df['slot_PsfFlux_instFlux'].map(convert_fluxtomag)

In [ ]:
#list(df.columns)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6))
df['base_PsfFlux_instFlux'].plot(kind="hist",bins=100,logx=True,logy=True,ax=ax1,grid=True)
df['slot_PsfFlux_instFlux'].plot(kind="hist",bins=100,logx=True,logy=True,ax=ax2,grid=True)
plt.suptitle("Detected Sources psf Fluxes " + exposure_title,fontsize=20)
ax1.set_xlabel('base_PsfFlux_instFlux')
ax2.set_xlabel('slot_PsfFlux_instFlux')
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6))
df['base_PsfFlux_instMag'].plot(kind="hist",bins=50,ax=ax1,logy=True,grid=True)
df['slot_PsfFlux_instMag'].plot(kind="hist",bins=50,ax=ax2,logy=True,grid=True)
ax1.set_xlabel('base_PsfFlux_instMag')
ax2.set_xlabel('slot_PsfFlux_instMag')
plt.suptitle("Detected Sources psf Magnitudes " + exposure_title,fontsize=20)
plt.tight_layout()

In [ ]:
calexp.info.getPhotoCalib()

In [ ]:
calexp.info.hasVisitInfo()

In [ ]:
calexp.info.getVisitInfo()

## Calibrate the Image

https://community.lsst.org/t/units-of-calexps/5998

In [ ]:
#calibrated = calexp.photoCalib.calibrateImage(calexp.maskedImage)

In [ ]:
# Display the cutout and sources with afw display
#image = cutout.image
#image = calibrated.image
#plt.figure()
#afw_display = afwDisplay.Display()
#afw_display.scale('asinh', 'zscale')
#afw_display.mtv(image, title= "Source Identification on Calibrated Calexp "+ exposure_title)
#plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image
#  after each source is plotted
#with afw_display.Buffering():
#    for s in sources:
#        afw_display.dot('.', s.getX(), s.getY(), ctype=afwDisplay.RED)
#        afw_display.dot('o', s.getX(), s.getY(), size=100, ctype='orange')

In [ ]:
#result.numPosPeaks